# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import sys
!{sys.executable} -m pip install pymystem3
!{sys.executable} -m pip install collections

ERROR: Could not find a version that satisfies the requirement collections
ERROR: No matching distribution found for collections


In [2]:
import pandas as pd
import numpy as np

from pymystem3 import Mystem
m = Mystem()
from collections import Counter

df = pd.read_csv('https://www.dropbox.com/s/n8cclxygfj4i5gr/data_prepearing.csv?dl=1')
print (df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


Выведем первые и последние 10 строк таблицы для первичной оценки форматов и однородности данных

In [3]:
display (df.head(10))
display (df.tail(10))


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


оценим строки с нулевыми значениями, согласно данным из метода .info() в столбцах с доходом и стажем

In [4]:

df['nana_income'] = df['total_income'].isnull() #column for counting NaN objects for creating new DataFrame by attribute
df['nana_days'] = df['days_employed'].isnull() #column for counting NaN objects for creating new DataFrame by attribute
skipped_values_income = df[df['nana_income'] == True] #creating new DataFrame by attribute NaN in clmn 'total income'
skipped_values_days = df[df['nana_days'] == True] #creating new DataFrame by attribute NaN in clmn 'days'
print ('пропущенных значений в столбце income -', len (skipped_values_income)) #counter number of NaN obj at column income
print ('пропущенных значений в столбце days_employe -', len (skipped_values_days)) #counter number of NaN obj at column days_employe
#print (skipped_values.head(30))
#print (skipped_values.tail(30))

#print (df)
skipped_income_types = skipped_values_days['income_type'].value_counts()


пропущенных значений в столбце income - 2174
пропущенных значений в столбце days_employe - 2174


**Вывод** 
 Пропущенные значения в строках 'total_income' и 'days_employe' встречаются одновременно, их число одинаково что может говорить о неслучайности пропусков, и возможной технической причине невозможности заполнить эти поля клиентами. Столбцы с доходом и отработанными днями - количественные, их значение может быть заменено средними или медианными величинами (доход - может соответствовать медиане по группе).

В информации всего 12 признаков по каждому заёмщику. Таблица содержит пропущенные значения в столбцах 'days_employed' и 'total_income'. Первичная оценка показывает визуальную неоднородность и некорректность некоторых данных (разные регистры, отрицательные значения в трудовом стаже)

## Шаг 2. Предобработка данных

### Обработка пропусков

***2.1.1 Анализ и заполнение столбца days_employed.***

Для заполнения пропущенных значений используем два показателя: **среднее арифм**. значение либо **медианное** по группировкам (профессия, возраст) 'days_employed'. 

Для первичного анализа проверим средние и медианные значения по всему столбцу days_employed:

In [5]:
print ('Среднее арифметическое по столбцу "трудовой стаж":', int(df['days_employed'].mean()), 
       'дней, или', int(df['days_employed'].mean()/365.25), 'года.' )
print ('Медианное значение по столбцу "трудовой стаж":', int(df['days_employed'].median()), 
       'дней, или', int(df['days_employed'].median()/365.25), 'года.' )

Среднее арифметическое по столбцу "трудовой стаж": 63046 дней, или 172 года.
Медианное значение по столбцу "трудовой стаж": -1203 дней, или -3 года.


Из полученных данных очевидно, что указанные **данные по столбцу** - **неоднородны и (или) некорректны** по своему формату или значению: 

1. в первом случае среднее арифм. значение превышающее возраст любого человека, живущего на Земле - 172 года;   
2. во втором случае стаж имеет отрицательное значение) -  (-3) года.  

заменим все отрицательные значения на противоположные по знаку:

In [6]:
def negative(days):
    if days < 0:
        positive = - days
        return positive
    else:
        positive = float(days)
        return positive
    
    
df['days_employed'] = df['days_employed'].apply(negative)

Вычислим и заменим экстремальные значения столбца 'days_employed'медианными. Для этого необходимо сравнить значение столбца с максимально возможным стажем (в днях). Для начала необходимо найти трудовой стаж. Для этого вычтем из возраста клиента минимально возможный легальный возраст начала трудовой деятельности - 14 лет. Умножим полученное значение на среднее количество дней в году (365,25). 

Сначала создадим столбец с максимально возможным легальным трудовым стажем

In [7]:
df['days_employed_max'] = (df['dob_years'] - 14)*365.25
df = df.astype({'days_employed_max': "float64"})
#df.info()


Оценим количество некорректных данных в столбце стажа 'days_employed', превышающих максимальные величины.

In [8]:
days_employed_uncorrect = df[df['days_employed'] > df['days_employed_max']]['days_employed'].count()
print ('Количество строк с превышение макс. возможного трудового стажа:', days_employed_uncorrect) 


Количество строк с превышение макс. возможного трудового стажа: 3534


3534 некорректных значений против 21525 строк всего (меньше половины) позволяет использовать медианное значение для замещения некорректных значений.

In [9]:
median_employed = df['days_employed'].median()
mean_employed = df['days_employed'].mean()
max_days_employed_oldest_client = df['days_employed_max'].max()
print ('Медиана по трудовому стажу -', int(median_employed), 'дней')
print ('Cредняя по трудовому стажу -', int(mean_employed), 'дней')
print ('Максимально возможный стаж самого взрослого клиента', int(max_days_employed_oldest_client), 'дней')

Медиана по трудовому стажу - 2194 дней
Cредняя по трудовому стажу - 66914 дней
Максимально возможный стаж самого взрослого клиента 22280 дней


Как видно средняя величина по стажу показывает наличие экстремальных значений. Заменим экстремальные значения на медианные

In [10]:

def extremal_days(row):
    days_employed = row['days_employed'] 
    days_employed_max = row['days_employed_max']  #предварительно создал столбец с максимально возможным стажем в днях 
    if days_employed > days_employed_max:
        return median_employed #возвращаю медиану если завышено значение
    else:
        return days_employed #возвращаю нормальное значение

df['days_employed'] = df.apply(extremal_days, axis = 1)

#print (df.head(10))



После замены экстремальных величин, медианные и средние значения стажа показывают следующие значения

In [11]:
median_employed = df['days_employed'].median()
mean_employed = df['days_employed'].mean()
print ('Медиана по трудовому стажу -', int(median_employed), 'дней')
print ('Cредняя по трудовому стажу -', int(mean_employed), 'дней')

Медиана по трудовому стажу - 2194 дней
Cредняя по трудовому стажу - 2317 дней


Заменим отсутствующие значения медианными **по группе занятости.**

Для начала определим медианные значения по группам занятости:

In [12]:
median_employee_days = df[df['income_type'] == 'сотрудник']['days_employed'].median()
median_companion_days = df[df['income_type'] == 'компаньон']['days_employed'].median()
median_retiree_days = df[df['income_type'] == 'пенсионер']['days_employed'].median()
median_gov_days = df[df['income_type'] == 'госслужащий']['days_employed'].median()
median_entrepreneur_days = df[df['income_type'] == 'предприниматель']['days_employed'].median()

print (median_retiree_days, median_employee_days)

2194.220566878695 1586.8763788059773


In [13]:
def filling_nan_days(row):
    daysemployed = row['days_employed']
    incometype = row['income_type']
    isnull_days = row['nana_days']
    
    if incometype == 'сотрудник' and isnull_days == True:
        return median_employee_days
    elif incometype == 'компаньон' and isnull_days == True:
        return median_companion_days
    elif incometype == 'пенсионер' and isnull_days == True:
        return median_retiree_days
    elif incometype == 'госслужащий' and isnull_days == True:
        return median_gov_days
    elif incometype == 'предприниматель' and isnull_days == True:
        return median_entrepreneur_days
    else:
        return daysemployed

df['days_employed'] = df.apply(filling_nan_days, axis = 1)

#df.info()
print (df['days_employed'].mean())
print (df['days_employed'].median())

2261.6785176953963
2010.6944743939828


**Вывод** по столбцу 'days_employed'

Данные по группам заменены, медиана и средняя показывают сопоставимые значения, замена критических значений стажа прошла успешно. Столбец 'days_employed' пригоден для дальнейшего анализа.


In [14]:
#df[df['income_type'] == 'сотрудник']['days_employed'].fillna(median_employee_days, inplace = True)
#df[df['income_type'] == 'пенсионер']['days_employed'].fillna(median_retiree_days, inplace = True)
#df[df['income_type'] == 'госслужащий']['days_employed'].fillna(median_gov_days, inplace = True)
#df[df['income_type'] == 'предприниматель']['days_employed'].fillna(median_entrepreneur_days, inplace = True)
#df[df['income_type'] == 'компаньон']['days_employed'].fillna(median_companion_days, inplace = True)

#ничего не заполнялось в df, хотя вроде inplace = True. Возможно я неправильно понимаю смысл этого метода или применяю не так

***2.1.2 Анализ столбца total_income.***

Оценим медианные и средние значения по доходу

In [15]:
print (df['total_income'].median())
print (df['total_income'].mean())

145017.93753253992
167422.3022081719


Заменим пропущенные значения медианными, соответствующими виду занятости. Ранее объявляли переменную skipped_income_type, отображающую вид занятости клиентов с пропущенными значениями

In [16]:
print (skipped_income_types)

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64


Создаём переменные - медианные значения дохода по видам деятельности. И далее создаём функцию для замены пропущенных значений. Ранее был создан столбец 'nana_income' - в котором содержатся булевы значения: TRUE - если строка содержит NaN в столбце total_income.

In [17]:
median_employee = df[df['income_type'] == 'сотрудник']['total_income'].median()
median_companion = df[df['income_type'] == 'компаньон']['total_income'].median()
median_retiree = df[df['income_type'] == 'пенсионер']['total_income'].median()
median_gov = df[df['income_type'] == 'госслужащий']['total_income'].median()
median_entrepreneur = df[df['income_type'] == 'предприниматель']['total_income'].median()

def income_median(rowd):
    income_type = rowd['income_type']
    nana_income = rowd['nana_income']
    income = rowd['total_income']
    
    if (nana_income == True) and (income_type == 'сотрудник'):
        return median_employee
    elif (nana_income == True) and (income_type == 'компаньон'):
        return median_companion
    elif (nana_income == True) and (income_type == 'пенсионер'):
        return median_retiree
    elif (nana_income == True) and (income_type == 'госслужащий'):
        return median_gov
    elif (nana_income == True) and (income_type == 'предприниматель'):
        return median_entrepreneur
    else:
        return income
        
df['total_income_right'] = df.apply(income_median, axis = 1)

    
df = df.drop(['nana_income', 'nana_days', 'days_employed_max'], axis = 1) #deleting extracolumns   


In [18]:
#print (df.info())

In [19]:
df['total_income'] = df['total_income_right']
df = df.drop(['total_income_right'], axis = 1) #deleting extracolumns   
#df.info() #cheking non-null objects

***2.1.3 Столбец 'children'***
Оценим столбец 'children' на некорректные значения

In [20]:
print ('before')
print (df['children'].value_counts())

before
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


Заменим отрицательные значения столбца 'children' на положительные, а также показатель 20 приведем  к значению 2 ввиду явной недостоверности. выведем результат

In [21]:
def children_negative (row):
    if row['children'] < 0:
        row['children'] = - row['children']
        return row['children']
    elif  row['children'] == 20:
        row['children'] = 2
        return row['children']
    else:
        return row['children']
print ('after')
df['children'] = df.apply(children_negative, axis = 1)
print (df['children'].value_counts())

after
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


**Вывод** пропущенные данные заполнены, таблица пригодна к дальнейшему анализу.

***2.1.4 анализ столбца 'dob_years'***

Проверим список уникальных значений столбца dob_years (возраст)

In [22]:
unique_years_old = df['dob_years'].value_counts()
unique_years_old.sort_index (axis = 0).head(5)

0     101
19     14
20     51
21    111
22    183
Name: dob_years, dtype: int64

In [23]:

print ('Всего нулей', df[df['dob_years'] == 0]['dob_years'].count())

print ('Пенсионеров с нулевым возрастом', df[(df['dob_years'] == 0) & (df['income_type'] == 'пенсионер')]['dob_years'].count())

Всего нулей 101
Пенсионеров с нулевым возрастом 20


Выявлено: 101 новорожденный заёмщик, из которых 20 родились пенсионерами. Меняем значения согласно среднему возрасту по группе семейное положение и типу занятости (пенсионер). **Логика следующая: если непенсионер  - применяем среднюю по возрасту согласно семейному положению, если пенсионер - средняя по пенсионерам.**

In [24]:
print (df['family_status'].value_counts())
print (df['family_status_id'].value_counts())

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64
0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64


In [25]:
#print (df[df['dob_years'] == 0]['income_type'].value_counts() )
#print (df[df['dob_years'] == 0]['gender'].value_counts() )
#print (df[df['dob_years'] == 0]['children'].value_counts() )
#print (df[df['dob_years'] == 0]['family_status'].value_counts() )
mean_years_not_married = df[df['family_status_id'] == 4]['dob_years'].mean()
mean_years_married = df[df['family_status_id'] == 0]['dob_years'].mean()
mean_years_divorced = df[df['family_status_id'] == 3]['dob_years'].mean()
mean_years_widow = df[df['family_status_id'] == 2]['dob_years'].mean()
mean_years_inlaw = df[df['family_status_id'] == 1]['dob_years'].mean()
mean_years_retire  = df[df['income_type'] == 'пенсионер']['dob_years'].mean()

def zero_years_replace(row):
    years_old  = row['dob_years']
    family_status_id = row['family_status_id']
    if (row['dob_years'] == 0) & (row['income_type'] != 'пенсионер') & (family_status_id == 0):
        return int(mean_years_married)
    elif (row['dob_years'] == 0) & (row['income_type'] != 'пенсионер') & (family_status_id == 1):
        return int(mean_years_inlaw)
    elif (row['dob_years'] == 0) & (row['income_type'] != 'пенсионер') & (family_status_id == 2):
        return int(mean_years_widow)
    elif (row['dob_years'] == 0) & (row['income_type'] != 'пенсионер') & (family_status_id == 3):
        return int(mean_years_divorced)
    elif (row['dob_years'] == 0) & (row['income_type'] != 'пенсионер') & (family_status_id == 4):
        return int(mean_years_not_married)
    elif (row['dob_years'] == 0) & (row['income_type'] == 'пенсионер'):
        return int(mean_years_retire)
    else: 
        return years_old
        
df['dob_years'] = df.apply(zero_years_replace, axis = 1)

print (df['dob_years'].unique())

[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51 59 29 60 55 58 71
 22 73 66 69 19 72 70 74 75]


### Замена типа данных

Тип данных float встречается в двух столбцах 'days_employed' и 'total_income' и 'dob_years'. Изменим их типы данных.

In [26]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df ['dob_years'] = df['dob_years'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Вещественные данные в датафрейме были заменены в столбцах 'days_employed' и 'total_income'. Замену производили с помощью метода astype() без предварительного округления. Дробная часть числа отработанных дней и доход составляет малую величину, не влияющую на общую картину исследования.

### Обработка дубликатов

При первичной оценке формата таблицы в столбце 'education' были замечены разные регистры. Приведём разнородные регистры к нижнему.

In [27]:
df['education'] = df['education'].str.lower() #set from uppercase to lowercase
df = df.drop_duplicates() 
#df.info()
#df['purpose'].value_counts()

**Вывод** дубликаты составили 71 единицы или 0,32% от числа строк. Исчезающе-малая величина в рамках задачи. Часть дубликатов выявлена после приведения регистра в нижний, что говорит о возможном повторном введении данных в датафрейм в ручном режиме (возможно со стороны клиента)

### Лемматизация

Приведём ячейку с целями кредита каждой строки к лемме, выведем леммы в отдельный столбец 'purpose_ex'. Используем функцию для строки и метод apply.

In [28]:
purpose = df['purpose']
#print (purpose)
summary_purpose = str(' ')


def text_maker_purpose ():
    summary_purpose = str(' ')
    for row in purpose:
        summary_purpose = summary_purpose + row + ' '
    return summary_purpose


text_purpose = text_maker_purpose()
lemmas = m.lemmatize (text_purpose)
 
counter_lemmas = Counter(lemmas)
display (counter_lemmas)

Counter({' ': 55021,
         'покупка': 5897,
         'жилье': 4460,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2323,
         'операция': 2604,
         'с': 2918,
         'на': 2221,
         'проведение': 767,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6351,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 853,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'получение': 1314,
         'высокий': 1374,
         'подержанный': 111,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607,
         ' \n': 1})

**Вывод** После проведённой лемматизации возможно визуально оценить основные группы (категории) целей займов. Всего 4 основные группы (по убыванию количества упоминаний): **недвижимость (жильё), автомобиль, образование, свадьба.**

### Категоризация данных

Для категоризации заёмщиков по целям, необходимо разбить их на укрупнённые группы. Воспользуемся массивом найденных лемм.
Для проверки частоты упоминаний различных целей займов воспользуемся счётчиком:

In [29]:
def subcategory_lemm (row):
    purpose_column = row['purpose']
    lemma_column = m.lemmatize(purpose_column)
    return lemma_column
df['subcategory_lemm']= df.apply(subcategory_lemm, axis = 1)
print (df['subcategory_lemm'])


0                             [покупка,  , жилье, \n]
1                   [приобретение,  , автомобиль, \n]
2                             [покупка,  , жилье, \n]
3                [дополнительный,  , образование, \n]
4                           [сыграть,  , свадьба, \n]
                             ...                     
21520                  [операция,  , с,  , жилье, \n]
21521               [сделка,  , с,  , автомобиль, \n]
21522                              [недвижимость, \n]
21523    [на,  , покупка,  , свой,  , автомобиль, \n]
21524             [на,  , покупка,  , автомобиль, \n]
Name: subcategory_lemm, Length: 21453, dtype: object


**свадьба - 2348.0    
недвижимость и жильё - 10840.0** из них коммерческая и под сдачу 1968.0, ремонт 612.0

**автокредит - 4315.0** и **образование - 4022.0**


Добавим столбец с укрупнёнными категориями целей займа

In [30]:
def purpose_cat(row):
    purpose = row['subcategory_lemm']
    if ('недвижимость' or 'жилье') and ('коммерческий' or 'сдача') in purpose:
        return 'недвижимость коммерческая'
    elif ('автомобиль') in purpose:
        return ('автокредит')
    elif ('ремонт') in purpose:
        return ('ремонт жилья')
    elif ('свадьба') in purpose:
        return ('свадьба')
    elif ('дополнительный') in purpose:
        return ('образование дополнительное')
    elif ('высокий') in purpose:
        return ('высшее образование')
    elif ('образование') in purpose:
        return ('образование')
       
    else:
        return ('недвижимость')
    
df['purpose_category'] = df.apply(purpose_cat, axis = 1)
print (df['purpose_category'])

0                      недвижимость
1                        автокредит
2                      недвижимость
3        образование дополнительное
4                           свадьба
                    ...            
21520                  недвижимость
21521                    автокредит
21522                  недвижимость
21523                    автокредит
21524                    автокредит
Name: purpose_category, Length: 21453, dtype: object


**Вывод** Датафрейм получил дополнительный столбец с укрупнёной категорией по целям займа. Данные готовы для решения задач.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Оценим долю лиц имевших просрочки к общему числу соответствующей группы

In [31]:
print ('Доля лиц с просрочками в зависимости от числа детей')
print (df.groupby('children')['debt'].sum() / df.groupby('children')['debt'].count())

Доля лиц с просрочками в зависимости от числа детей
children
0    0.075444
1    0.091658
2    0.094925
3    0.081818
4    0.097561
5    0.000000
Name: debt, dtype: float64


Как видно, зависимость есть, однако наличие детей у лиц старшего и младшего возраста может влиять на просрочки по-разному. Проверим эту гипотезу. Для этого определим медианный возраст, затем определим долю лиц с просрочками с возрастом ниже и выше медианного. Просрочки у лиц, чьё число детей составляет 20 - вероятно говорит о присутствии заведомо ложной информации, представленной в кредитное учреждение.

In [32]:
print ('Медианный возраст (лет)', df['dob_years'].median())

print ('Доля клиентов с просрочками в возрасте ниже медианного:')
df_lower_median_dob = df[df['dob_years'] < 43.0]
print (df_lower_median_dob.groupby('children')['debt'].sum() / df_lower_median_dob.groupby('children')['debt'].count())
print ('________________________________________________')
print ('Доля клиентов с просрочками в возрасте выше медианного:')
df_upper_median_dob = df[df['dob_years'] > 43.0]
print (df_upper_median_dob.groupby('children')['debt'].sum() / df_upper_median_dob.groupby('children')['debt'].count())

Медианный возраст (лет) 43.0
Доля клиентов с просрочками в возрасте ниже медианного:
children
0    0.096072
1    0.100825
2    0.098536
3    0.090253
4    0.058824
5    0.000000
Name: debt, dtype: float64
________________________________________________
Доля клиентов с просрочками в возрасте выше медианного:
children
0    0.062806
1    0.067424
2    0.066434
3    0.051282
4    0.285714
5    0.000000
Name: debt, dtype: float64


Как видно, старшая возрастная категория с 4мя детьми обладает крайне высокими значениями. Оценим отдельно эту категорию заёмщиков.

In [33]:
children_four_old = df[(df['children'] == 4) & (df['dob_years'] > 42)]
children_four_old['purpose'].value_counts()

на покупку автомобиля                     2
строительство собственной недвижимости    1
покупка жилой недвижимости                1
свой автомобиль                           1
заняться высшим образованием              1
получение дополнительного образования     1
Name: purpose, dtype: int64

**Вывод** наличие детей влияет на процент просрочек, но в меньшей степени на старшую возрастную категорию, что вероятно обусловлено недостоверной информацией. Среди реальных групп наибольшее число просрочек наблюдается у группы с **четырьмя детьми**, причём у старшей возрастной категории доля невозвратов составляет 28,57% против 8 - 10% в остальных категориях, что может быть обусловлено как **малой выборкой так и социально-экономическим профилем заёмщиков этой категории**.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [34]:
print ('Доля клиентов с просрочками в возрасте выше медианного:')
print (df.groupby('family_status')['debt'].sum() / df.groupby('family_status')['debt'].count())
print (df.groupby('family_status')['dob_years'].mean())

Доля клиентов с просрочками в возрасте выше медианного:
family_status
Не женат / не замужем    0.097509
в разводе                0.071130
вдовец / вдова           0.065693
гражданский брак         0.093494
женат / замужем          0.075452
Name: debt, dtype: float64
family_status
Не женат / не замужем    38.609964
в разводе                45.917992
вдовец / вдова           56.811262
гражданский брак         42.286024
женат / замужем          43.728260
Name: dob_years, dtype: float64


**Вывод** Очевидно, чем чаще человек посещает ЗАГС, и чем он старше тем ниже вероятность просрочки по кредиту. 
Снижение доли просрочек после вступления в брак может быть объяснено созданием общего семейного бюджета. 

<div class="alert alert-success">
<b>Комментарий ревьюера: </b> Интересная интерпретация! :) </div>

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Рассмотрим средние доходы по видам занятости. 

In [35]:
df.groupby('income_type')['total_income'].mean().sort_values(axis = 0, ascending = True)



income_type
в декрете           53829.000000
студент             98201.000000
безработный        131339.000000
пенсионер          135250.604597
сотрудник          159567.794280
госслужащий        168862.543583
компаньон          199451.225089
предприниматель    499163.000000
Name: total_income, dtype: float64

Виды занятости - определяющим образом влияют на уровень дохода. Проверим гипотезу зависимости возврата кредита в срок и вида занятости.

In [36]:
print (df['income_type'].value_counts())

print ('________________________________________________________________')
print ('Доля клиентов с просрочками в зависимости от типа занятости (уровня дохода):')
part_debt_negative = df.groupby('income_type')['debt'].sum() / df.groupby('income_type')['debt'].count()
#print (part_debt_negative.sort_values(axis = 0, ascending = True))
print (part_debt_negative)

сотрудник          11083
компаньон           5078
пенсионер           3829
госслужащий         1457
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64
________________________________________________________________
Доля клиентов с просрочками в зависимости от типа занятости (уровня дохода):
income_type
безработный        0.500000
в декрете          1.000000
госслужащий        0.059025
компаньон          0.074045
пенсионер          0.056412
предприниматель    0.000000
сотрудник          0.095732
студент            0.000000
Name: debt, dtype: float64


Как видно - выборка по предпринимателям, безработным, студентам, и декретным - крайне мала и не репрезентативна. Введём новый столбец, с градацией дохода: 'низкий', 'ниже среднего', 'выше среднего', 'высокий' с границами по процентилям 25, 50, 75.

In [37]:
#mean_income_total = int(df['total_income'].mean())
#print (df['total_income'].describe())

def range_income(row):
    total_income_r = row['total_income']
    if total_income_r > 195820:    
        return 'высокий доход'
    elif 142594 < total_income_r < 195820:
        return 'доход выше среднего'
    elif 107623 < total_income_r < 142594:
        return 'доход ниже среднего'
    else:
        return 'низкий доход'
    
df['category_of_income'] = df.apply(range_income, axis = 1)


Распечатаем долю клиентов с просрочками в зависимости от уровня дохода.

In [38]:
print ('Доля клиентов с просрочками в зависимости от уровня дохода:')
print (df.groupby('category_of_income')['debt'].sum() / df.groupby('category_of_income')['debt'].count())

Доля клиентов с просрочками в зависимости от уровня дохода:
category_of_income
высокий доход          0.071402
доход выше среднего    0.085382
доход ниже среднего    0.087321
низкий доход           0.081610
Name: debt, dtype: float64


**Вывод** Оценка зависимости уровня дохода от просрочек показала незначительное влияние уровня дохода от доли клиентов с просрочками. ***При этом клиенты с высоким доходом реже отдают кредиты с просрочками (показатель меньше на 1.4%).***

- Как разные цели кредита влияют на его возврат в срок?

In [39]:
print ('Доля просрочек в зависимости от целей кредита:')
df.groupby('purpose_category')['debt'].sum() / df.groupby('purpose_category')['debt'].count()

Доля просрочек в зависимости от целей кредита:


purpose_category
автокредит                    0.093590
высшее образование            0.093886
недвижимость                  0.072866
недвижимость коммерческая     0.075515
образование                   0.087709
образование дополнительное    0.098234
ремонт жилья                  0.057661
свадьба                       0.080069
Name: debt, dtype: float64

**Вывод** 
Наименьшая вероятность просрочки возникает при кредитах под ремонт недвижимости. Максимальная вероятность - получение дополнительного образования.

Цель кредита для операций с коммерческой недвижимостью или жильём под сдачу в меньшей степени влияет на возврат кредита в срок по причине более вероятного возникновения дополнительного источника дохода в виде ренты.

## Шаг 4. Общий вывод

Наличие детей влияет на просрочки по кредитам в большей степени в молодой категории заёмщиков, при этом в старшей возрастной категории группа с 4мя детьми показывает экстремальный результат, это может быть вызвано малой выборкой а также профилем дохода многодетных семей с взрослыми родителями. Бездетные заёмщики обеспечивают самые низкие показатели просрочек - 7,5%, следом идут заёмщики с 3 мя детьми 8,2%, заёмщики с 5ю детьми исходя из данных просрочек не имеют, что вероятно обусловлено малым числом выборки (всего 9 заёмщиков). 
<ul>
<li>0  детей   7,5% просрочек</li>
<li>1 ребёнок 9,17% просрочек</li>
<li>2 ребёнка 9,5% просрочек</li>
<li>3 ребёнка 8,2% просрочек</li>
<li>4 ребёнка 9,8% просрочек</li>

</ul>

Что касается семейного положения - его влияние на процент просрочек более очевидно:


<ul>
<li>вдовец / вдова 6,5%</li>
<li>в разводе 7,1%</li>
<li>женат / замужем  7,5%</li>
<li>гражданский брак 9,3%</li>
<li>Не женат / не замужем 9,7%</li>

</ul>

Наиболее добросовестными заёмщиками являются вдовы и вдовцы, менее добросовестными - холостые. Этот критерий в большей степени подходит для расчёта скоринга.




Вид занятости также отражается на процессе погашения кредита: наилучшими показателями обладают госслужащие и пенсионеры, что вероятнее всего обусловлено большей предсказуемостью их дохода по сравнению с иными категориями.

<ul>
<li>пенсионер     5,6% просрочек</li>
<li>госслужащий   5,9% просрочек</li>
<li>компаньон    7,4% просрочек</li>
<li>сотрудник     9,5% просрочек</li>
</ul>


Анализ просрочек по целям кредита позволяет заметить следующее:

<ul>
<li>ремонт жилья                  5,8%</li>
<li>недвижимость жилая            7,3%</li>
<li>недвижимость коммерческая     7,6%</li>
<li>свадьба                       8,0%</li>
<li>образование                   8,8%</li>
<li>автокредит                    9,4%</li>
<li>высшее образование            9,4%</li>
<li>образование дополнительное    9,8%</li>
</ul>


Недвижимость (покупка, ремонт) является одной из самых "надёжных целей" получения кредита, что вероятнее во много связано с изменением семейного положения заёмщика.
